In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_csv('training_images/training_images.csv')

In [29]:
df
X = df.drop('label', axis=1)
y = pd.get_dummies(df['label'])

In [30]:
model=Sequential()
model.add(Dense(64, input_dim=1024, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(36, activation='softmax'))


In [31]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
model.fit(X_train, y_train,validation_data=(X_test,y_test), epochs=20)

Epoch 1/20
1800/1800 [==============================] - 4s 2ms/step - loss: 0.2394 - accuracy: 0.9260 - val_loss: 0.4338 - val_accuracy: 0.8835
Epoch 2/20
1800/1800 [==============================] - 3s 2ms/step - loss: 0.2235 - accuracy: 0.9302 - val_loss: 0.4323 - val_accuracy: 0.8847
Epoch 3/20
1800/1800 [==============================] - 3s 2ms/step - loss: 0.2132 - accuracy: 0.9336 - val_loss: 0.4261 - val_accuracy: 0.8874
Epoch 4/20
1800/1800 [==============================] - 4s 2ms/step - loss: 0.2018 - accuracy: 0.9364 - val_loss: 0.4609 - val_accuracy: 0.8785
Epoch 5/20
1800/1800 [==============================] - 4s 2ms/step - loss: 0.1887 - accuracy: 0.9411 - val_loss: 0.4284 - val_accuracy: 0.8870
Epoch 6/20
1800/1800 [==============================] - 4s 2ms/step - loss: 0.1802 - accuracy: 0.9427 - val_loss: 0.4559 - val_accuracy: 0.8824
Epoch 7/20
1800/1800 [==============================] - 4s 2ms/step - loss: 0.1732 - accuracy: 0.9452 - val_loss: 0.4727 - val_accuracy:

In [35]:
import pickle

# Serialize the model to a pkl file
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

INFO:tensorflow:Assets written to: ram://85c51ba4-72a5-4c86-a45e-a94ee1110785/assets
